# Lab | Inferential statistics - T-test & P-value


In [1]:
import pandas as pd
import numpy as np
import scipy.stats as st
import matplotlib.pyplot as plt
import seaborn as sns

One-tailed t-test - In a packing plant, a machine packs cartons with jars. It is supposed that a new machine will pack faster on average than the machine currently used. To test that hypothesis, the times each machine takes to pack ten cartons are recorded. The results are in seconds in the tables in the file files_for_lab/machine.txt. Assume that there is sufficient evidence to conduct the t-test, does the data provide sufficient evidence to show if one machine is better than the other?

Null hypothesis (H0): The new machine does not pack faster on average than the machine currently used. In terms of the population means, it can be expressed as:

H0: mean_new_machine >= mean_current_machine

Alternative hypothesis (H1): The new machine packs faster on average than the machine currently used. This can be expressed as:

H1: mean_new_machine < mean_current_machine

where:
mean_new_machine- is the population mean time taken by the new machine to pack ten cartons.
mean_current_machine- is the population mean time taken by the machine currently in use to pack ten cartons.

In [2]:
machine = pd.read_csv('machine.txt',delimiter='\t', encoding='utf-16')

In [3]:
machine.head()

,New machine,Old machine
0,42.1,42.7
1,41.0,43.6
2,41.3,43.8
3,41.8,43.3
4,42.4,42.5


In [4]:
machine.columns

Index(['New machine', '    Old machine'], dtype='object')

In [5]:
# Extract the columns for the new and old machine
new_machine_times = machine['New machine']
old_machine_times = machine['    Old machine']

# Compute the means for both machines
mean_new_machine = new_machine_times.mean()
mean_old_machine = old_machine_times.mean()

# Perform one-tailed t-test assuming unequal variances
t_stat, p_value = st.ttest_ind(new_machine_times, old_machine_times, equal_var=False)

# Define the significance level
alpha = 0.05
#print t_stat and p_value
print("tstat-", t_stat)
print("p_value-", p_value)
# Check if the p-value is less than the significance level
if p_value < alpha and mean_new_machine < mean_old_machine:
    print("Reject the null hypothesis.")
else:
    print("Fail to reject the null hypothesis.")

tstat- -3.3972307061176026
p_value- 0.00324224946631798
Reject the null hypothesis.


p-value is less than the significance level of 0.05 and the mean time taken by the new machine is less than the mean time taken by the old machine, we reject the null hypothesis. Therefore, there is sufficient evidence to suggest that the new machine packs faster than the old machine.

Matched Pairs Test - In this challenge we will compare dependent samples of data describing our Pokemon (file files_for_lab/pokemon.csv). Our goal is to see whether there is a significant difference between each Pokemon's defense and attack scores. Our hypothesis is that the defense and attack scores are equal. Compare the two columns to see if there is a statistically significant difference between them and comment on your result.

In [6]:
pokemon = pd.read_csv('pokemon.csv')

In [7]:
pokemon.head()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


Null hypothesis (H0): There is no significant difference between each Pokemon's defense and attack scores In terms of the population means, it can be expressed as:

H0: mean_defense = mean_attack

Alternative hypothesis (H1): There is no significant difference between each Pokemon's defense and attack scores In terms of the population means, it can be expressed as:

H0: mean_defense != mean_attack


In [8]:
pokemon_attack_defense = pokemon[['Attack' , 'Defense']]

In [9]:
pokemon_attack_defense.head()

,Attack,Defense
0,49,49
1,62,63
2,82,83
3,100,123
4,52,43


In [10]:
#selecting sample
sample = pokemon_attack_defense.sample(30, random_state = 1)
sample['difference'] = sample['Attack']-sample['Defense']
sample.head()

,Attack,Defense,difference
8,104,78,26
510,92,75,17
175,46,34,12
735,50,58,-8
242,105,75,30


In [11]:
#calculating mean and std
sample_diff_mean, sample_diff_std = sample['difference'].mean(), sample['difference'].std(ddof=1)
sample_diff_mean, sample_diff_std

(2.8, 36.32079294288603)

In [12]:
#computing tstat

In [13]:
t = sample_diff_mean / ( sample_diff_std / np.sqrt(sample.shape[0]) )
print("The mean of our samples differences is: {:.2f}".format(sample_diff_mean))
print("The standard deviation of our samples differences is: {:.2f}".format(sample_diff_std))
print("Our t statistics is: {:.2f}".format(t))

The mean of our samples differences is: 2.80
The standard deviation of our samples differences is: 36.32
Our t statistics is: 0.42


In [14]:
#computing critical value
tc = st.t.ppf(1-(0.05/2),df= sample.shape[0] - 1)
tc

2.045229642132703

Our statistic is 0.42 while the critical value is 2.045. Then, as 0.42 < 2.05 we accept the H0.
t-statistic is much smaller than the critical value. This implies that we fail to reject the null hypothesis. 

In [15]:
#With p_values
1-st.t.cdf(t,df = sample.shape[0] - 1)

0.3379805871158601

since p value is graeter than 0.05 ,this suggests that there is no significant difference between the Attack and Defense scores of the Pokemon. therefore we accept null hypothesis

In [16]:
#using scipy
st.ttest_rel(sample['Attack'],sample['Defense'])

TtestResult(statistic=0.4222438544846934, pvalue=0.6759611742317202, df=29)

In [17]:
1-st.t.cdf(t,df = sample.shape[0] - 1)

0.3379805871158601

the p-value is 0.3379805871158601, which is greater than the typical significance level of 0.05, therefore we accept null hypothesis

# Inferential statistics - ANOVA

In this activity, we will look at another example. Your task is to understand the problem and write down all the steps to set up ANOVA. - Null hypothesis - Alternate hypothesis - Level of significance - Test statistic - P-value - F table

Suppose you are working as an analyst in a microprocessor chip manufacturing plant. You have been given the task of analyzing a plasma etching process with respect to changing Power (in Watts) of the plasma beam. Data was collected and provided to you to conduct statistical analysis and check if changing the power of the plasma beam has any effect on the etching rate by the machine. You will conduct ANOVA and check if there is any difference in the mean etching rate for different levels of power. You can find the data anova_lab_data.xlsx file in the files_for_lab folder

State the null hypothesis
State the alternate hypothesis
What is the significance level
What are the degrees of freedom of the model, error terms, and total DoF
Data were collected randomly and provided to you in the table as shown: link to the image - Data

In [18]:
anova_data = pd.read_excel('anova_lab_data.xlsx')

In [19]:
anova_data.head()

,Power,Etching Rate
0,160 W,5.43
1,180 W,6.24
2,200 W,8.79
3,160 W,5.71
4,180 W,6.71


State the null hypothesis-
Null Hypothesis:The mean etching rate is same for different levels of power.
State the alternate hypothesis -
Alternate Hypothesis:The mean etching rate is not same for different levels of power
What is the significance level?
0.05



What are the degrees of freedom of the model, error terms, and total DoF

In [20]:
anova_data.columns

Index(['Power ', 'Etching Rate'], dtype='object')

In [21]:
group_df = anova_data.groupby('Power ')['Etching Rate'].agg( Totals='sum' , Averages='mean').reset_index()
group_df

,Power,Totals,Averages
0,160 W,28.96,5.792
1,180 W,31.19,6.238
2,200 W,41.59,8.318


In [22]:
d1 = anova_data['Power '].nunique() - 1    # model terms
d2 = len(anova_data) - anova_data['Power '].nunique()  # error terms
total_dof = len(anova_data) - 1                          # total dof
print("Number of degrees of freedom d1: ",d1)
print("Number of degrees of freedom d2: ",d2)
print("Total degrees of freedom: ", total_dof)

Number of degrees of freedom d1:  2
Number of degrees of freedom d2:  12
Total degrees of freedom:  14


# Part 2

using scipy.stats includes a function ANOVA

In [24]:
from scipy.stats import f_oneway

f_stat, p_val = f_oneway(anova_data[anova_data['Power '] == '160 W']['Etching Rate'],
                         anova_data[anova_data['Power '] == '180 W']['Etching Rate'],
                         anova_data[anova_data['Power '] == '200 W']['Etching Rate'])
print(f_stat, p_val)

36.87895470100505 7.506584272358903e-06


using python

In [25]:
# In our case, the groups are the power, and the "y" is the "Etching Rate"
S2t = 0
for power in anova_data['Power '].unique():
    ng = len(anova_data[anova_data['Power '] == power])
    S2t  += ng * ( ( anova_data[anova_data['Power '] == power]['Etching Rate'].mean() - anova_data['Etching Rate'].mean() ) ** 2)
S2t /= ( anova_data['Power '].nunique() - 1 ) # Number of  K = groups/cities

print("The value of S2t is {:.2f}".format(S2t))

The value of S2t is 9.09


In [26]:
S2E = 0
for power in anova_data['Power '].unique():
    for rate in anova_data[anova_data['Power '] == power]['Etching Rate']:
        S2E += ( rate - anova_data[anova_data['Power '] == power]['Etching Rate'].mean() ) ** 2
S2E /= ( len(anova_data) - anova_data['Power '].nunique() )

print()
print("The value of S2E is {:.2f}".format(S2E))


The value of S2E is 0.25


In [27]:
S2E2 = 0
for power in anova_data['Power '].unique():
    group_data = anova_data[anova_data['Power '] == power]['Etching Rate']
    ng = len(group_data)
    S2E2 += (ng - 1) * np.var(group_data, ddof=1)
S2E2 /= (len(anova_data) - anova_data['Power '].nunique())

print("The value of S2E is {:.2f}".format(S2E2))

The value of S2E is 0.25


In [28]:
#compute F:
F = S2t / S2E
print("The value of F is {:.2f}".format(F))

The value of F is 36.88


In [29]:
st.f.cdf(F,dfn=d1, dfd=d2)

0.9999924934157276

In [30]:
1 - st.f.cdf(F,dfn=d1, dfd=d2)

7.5065842723986975e-06

both the methods give f value as 36.88 and probability as v small
As the p_value is < 0.05 we reject the H0.